In [1]:
import sys
sys.path.append('../saerch')
import family
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matrix
import plotly.io as pio
%load_ext autoreload
%autoreload 2

/Users/christineye/anaconda3/envs/jsalt-retrieval/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/christineye/anaconda3/envs/jsalt-retrieval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# feature family heatmap

In [32]:
mode = 'epoch_astroPH_old'
SAE_DATA_DIR = '../saerch/sae_data_{}/'.format('astroPH')

model_16_a = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/16_3072_24_auxk_{}_50.pth'.format(mode), autointerp_results = 'feature_analysis_results_16.json',)
model_32_a = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/32_6144_48_auxk_{}_50.pth'.format(mode),  autointerp_results = 'feature_analysis_results_32.json',)
model_64_a = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/64_9216_128_auxk_{}_50.pth'.format(mode),autointerp_results = 'feature_analysis_results_64.json',)

../saerch/sae_data_astroPH/topk_indices_16_3072.npy
271494 3072
271494 3072
../saerch/sae_data_astroPH/topk_indices_32_6144.npy
271494 6144
271494 6144
../saerch/sae_data_astroPH/topk_indices_64_9216.npy
271494 9216
271494 9216


In [36]:
families_64_a = model_64_a.load_all_families()

In [2]:
mode = 'csLG_epoch'
SAE_DATA_DIR = '../saerch/sae_data_{}/'.format('csLG')
model_64_c = family.Model(sae_data_dir = SAE_DATA_DIR, model_path = '../models/64_9216_128_auxk_{}_88.pth'.format(mode), autointerp_results = 'feature_analysis_results_64.json')
families_64 = model_64_c.load_all_families()

../saerch/sae_data_csLG/topk_indices_64_9216.npy
271494 9216
271494 9216


In [9]:
def make_single_heatmap(parent, model, display = 'cooc'):
    all_names, ids = parent.get_names_and_ids(model)
    
    norm_mat, ratio = parent.matrix(model, mode = display, return_mat = True)

    np.fill_diagonal(norm_mat, 1)

    heatmap = go.Heatmap(
        z=norm_mat,
        x=all_names,
        y=all_names,
        hoverongaps=False,
        hovertemplate='Name: %{x} <br> %{y} <br> %{z}'
    )

    # Create the figure
    fig = go.Figure(data=[heatmap])

    # Update layout for better readability and show only parent labels
    fig.update_layout(
        width=680,
        height=600,
        yaxis = dict(scaleanchor = 'x'),
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=20, r=20, t=20, b=20),
    )
    
    fig.show()
    

In [27]:
families_64['Odor Identification with Electronic Noses'].children

[['Facebook-specific research and applications', 1673],
 ['Spam detection using machine learning', 1793],
 ['ML/AI in network management and security', 2815],
 ['Thermal imaging with deep learning', 3385],
 ['Peaks in scientific contexts', 3962],
 ['Imperfect Information Games and RL', 4971],
 ['Fall detection and prediction systems', 8074],
 ['Sepsis treatment using reinforcement learning', 8893]]

In [28]:
make_single_heatmap(families_64['Odor Identification with Electronic Noses'], model_64_c)

In [38]:
families_64_a['Machine learning'].children

[['Convolutional Neural Networks', 180],
 ['Deep learning', 725],
 ['Artificial Neural Networks', 1446],
 ['Generative Adversarial Networks (GANs)', 5270],
 ['Regression analysis', 7802],
 ['Use of autoencoders', 4538],
 ['Automation and Machine Learning', 610],
 ['Histograms in Data Analysis', 3495],
 ['Denoising techniques', 1824],
 ['Real-time detection of transient events', 5909],
 ['Citizen science in Galaxy Zoo', 8932],
 ['Artificial Intelligence', 5709],
 ['Astrophysics Data System (ADS)', 2381],
 ['Gaussian Processes', 1019],
 ['Prediction/Forecasting', 6046]]

In [39]:
def make_big_heatmap(family_names, families, model, save = False, display ='cooc'):
    all_ids = []
    all_names = []

    for parent in family_names:
        children = [child[0] for child in families[parent].children]
        ids = [child[1] for child in families[parent].children]
        densities = [model.norms[id] for id in ids]
        
        ids = list(np.array(ids)[np.argsort(densities)])
        ids.append(model.clean_labels[parent]['index'])
        all_ids.append(ids)
        all_names.append(children + [parent])
    
    all_ids = [ind for sublist in all_ids for ind in sublist]
    
    # create matrix, normalized by the minimum of the norms of the two features
    if display == 'cooc':
        min_matrix = np.minimum.outer(model.norms, model.norms)
        norm_mat = (model.mat / min_matrix)[all_ids][:, all_ids]
    else:
        min_matrix = 1
        norm_mat = (model.actsims / min_matrix)[all_ids][:, all_ids]
    np.fill_diagonal(norm_mat, 1)

    heatmap = go.Heatmap(
        z=norm_mat,
        hoverongaps=False,
        hovertemplate='Name: %{x} <br> %{y} <br> %{z}'
    )

    # Create the figure
    fig = go.Figure(data=[heatmap])

    # Update layout for better readability and show only parent labels
    fig.update_layout(
        width=480,
        height=400,
        margin=dict(l=20, r=20, t=20, b=20),
    )
    fig.update_yaxes(showticklabels=False)
    fig.update_xaxes(showticklabels=False)
    fig.show()
    
    if save: fig.write_image("cooc_grid.png",  )
    


In [44]:
list(families_64_a.keys())[:15]

['Lyman-alpha and Lyman-beta emissions',
 'Machine learning',
 'Age determination of stellar populations',
 'Polarization',
 'High-energy astrophysics and particle physics',
 'Exoplanets and their characteristics',
 'Gas dynamics',
 'Black holes',
 'CMB, Polarization, Quasars, Magnetic Reconnection',
 'Gravitational waves and their detection',
 'Hayabusa mission and asteroid Itokawa',
 'Am stars (metallic-line A stars)',
 'Periodicity detection methods',
 'Binary star systems and their evolution',
 'Coronal phenomena']

In [47]:
make_big_heatmap(['Machine learning','Gamma-Ray Bursts (GRBs)', 'Black holes', 'Periodicity detection methods', 'Gas dynamics'], families_64_a, model_64_a, display = 'cooc')

# feature family interpretability

In [48]:
from autointerp import NeuronAnalyzer
analyzer = NeuronAnalyzer('../config.yaml', 691, 5)

def interp_superfeature(family, model):
    all_names, ids = family.get_names_and_ids(model)
    superfeature = analyzer.generate_family_feature(all_names[-1], all_names[:-1])
    print(all_names[-1], superfeature)

    top_abstracts, zero_abstracts = analyzer.get_feature_activations(5, feature_index = ids)

    num_test_samples = 4
    test_abstracts = [abstract for _, abstract, _ in top_abstracts[-num_test_samples:] + zero_abstracts[-num_test_samples:]]
    ground_truth = [1] * num_test_samples + [0] * num_test_samples

    predictions = analyzer.predict_activations(superfeature, test_abstracts)
    correlation, f1 = analyzer.evaluate_predictions(ground_truth, predictions)

    print(f"Pearson correlation: {correlation}")
    print(f"F1 score: {f1}")

    feature_pearsons = [model.clean_labels[feature]['pearson_correlation'] for feature in all_names]
    
    return {'superfeature': superfeature, 'family_f1': f1, 'feature_f1': feature_pearsons, 'feature_names': all_names}

In [50]:
interp_superfeature(families_64_a['Gas dynamics'], model_64_a)

Gas dynamics Interstellar and Intergalactic Gas Processes


100%|██████████| 8/8 [00:47<00:00,  5.94s/it]

Pearson correlation: 0.7533473197396057
F1 score: 0.8888888888888888


{'superfeature': 'Interstellar and Intergalactic Gas Processes',
 'family_f1': 0.8888888888888888,
 'feature_f1': [0.732101925434405,
  0.9271726499455307,
  0.6790180953188891,
  0.7435897435897438,
  0.7518820606756285,
  0.7602859212697056,
  0.9762956279494205,
  0.8780541105074453,
  0.7071067811865477,
  0.9883547702893383,
  0.6607008494562122,
  0.6155870112510924],
 'feature_names': ['Molecular gas in high redshift galaxies',
  'Circumgalactic Medium (CGM)',
  'Galaxy Interactions and Mergers',
  'Ram pressure stripping in galaxies',
  'Fueling mechanisms in galaxies',
  'Gas expulsion in star clusters',
  'Galactic fountains and water fountain nebulae',
  'H I (neutral hydrogen) emission in galaxies',
  'Cloud-based science platforms and technologies',
  'HI (Neutral Atomic Hydrogen) in Galaxies',
  'Infall motion in star formation',
  'Gas dynamics']}

In [ ]:
def create_radar_chart(feature_f1, overall_f1, feature_names, showlegend=False):
    # Add the first feature name to the end to close the polygon
    categories = feature_names + [feature_names[0]]
    feature_f1 = feature_f1 + [feature_f1[0]]
    
    return [
        go.Scatterpolar(
            r=feature_f1,
            theta=categories,
            fill='toself',
            name='Individual Features',
            line=dict(color='rgb(31, 119, 180)', width=5),
            showlegend=showlegend
        ),
        go.Scatterpolar(
            r=[overall_f1] * len(categories),
            theta=categories,
            name='Family F1 (base)',
            line=dict(color='red', width=5, dash='dash'),
            showlegend=showlegend
        ),
    ]

def plot_radar_charts(all_results):
    n_cols = 2
    n_rows = 2

    fig = make_subplots(
        rows=n_rows, cols=n_cols, 
        specs=[[{'type': 'polar'}] * n_cols] * n_rows,
        subplot_titles=list(all_results.keys()),
        vertical_spacing=0.15
    )

    for i, family in enumerate(all_results):
        row = i // n_cols + 1
        col = i % n_cols + 1
        
        family_results = all_results[family]
        radar_traces = create_radar_chart(family_results['feature_f1'], family_results['family_f1'], family_results['feature_names'], showlegend=(i==0))
        
        for trace in radar_traces:
            fig.add_trace(trace, row=row, col=col)

    # Update layout
    fig.update_layout(
        height=2000,  # Increased height to accommodate larger fonts
        width=2700,   # Increased width for better spacing
        font=dict(family="Palatino", size=12),  # Base font size
        paper_bgcolor='white',
        plot_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.12,
            xanchor="right",
            x=1,
            font=dict(size=36)  # Legend font size
        )
    )

    # Update polar axes to ensure consistent scaling and increase label size
    fig.update_polars(
        radialaxis=dict(
            range=[0, 1],
            tickfont=dict(size=12),
            tickangle=45
        ),
        angularaxis=dict(
            tickfont=dict(size=17)
        )
    )

    # Adjust subplot titles position and font size
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=36, family="Palatino")
        i['y'] = i['y'] + 0.05  # Move title up further

    # Save figure

    fig.show()
    fig.write_image('radar_charts.pdf')


In [ ]:
all_results = interp_superfeatures(['Machine learning', 'Gravitational waves and their detection', 'Gamma-Ray Bursts (GRBs)', 'Accretion processes'], families, model_64)
plot_radar_charts()